In [7]:
import re
import pandas as pd
import numpy as np

In [22]:
def startsWithDateAndTime(s):
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]) ([0-9]+):([0-9][0-9]) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [72]:
def getDataPoint(line):   
    splitLine = line.split(' - ')
    dateTime = splitLine[0]
    date, time = dateTime.split(' ') 
    message = ' '.join(splitLine[1:])
    splitMessage = message.split(': ')
    author = splitMessage[0]
    message = ' '.join(splitMessage[1:])
    return date, time, author, message

In [73]:
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = '../data/Chat de WhatsApp con Vir.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip()
        if startsWithDateAndTime(line):
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

In [74]:
df.head()

,Date,Time,Author,Message
0,2019-12-16,20:28,Vir,Mañana nos juntamos media horita y vemos un po...
1,2019-12-16,20:28,Vir,?!
2,2019-12-16,20:28,Vir,Un poco a modo de recuperar las weekly que ten...
3,2019-12-16,21:10,Leticia Olábarri,"Te los agradecería muchísimo, la verdad"
4,2019-12-16,21:11,Leticia Olábarri,*lo


In [75]:
df.shape

(7091, 4)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7091 entries, 0 to 7090
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     7091 non-null   datetime64[ns]
 1   Time     7091 non-null   object        
 2   Author   7091 non-null   object        
 3   Message  7091 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 221.7+ KB


In [78]:
df.isnull().sum()

Date       0
Time       0
Author     0
Message    0
dtype: int64

In [79]:
df.to_csv("../data/Vir.csv", index=False, header=True)